In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import wrds
import pandas as pd
import numpy as np
import statsmodels.api as sm

# import collections
# import torch
# from torch import nn
# from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
#Benchmarking PCA model

# GET Input Feature Data

In [123]:
zacks_fc = pd.read_csv("../data/fundamental_data/ZACKS_FC.txt", delimiter=',')
zacks_fc.head()
# zacks_fc['per_end_date'] = pd.to_datetime(zacks_fc['per_end_date'])
# zacks_fc['per_end_date'] = zacks_fc['per_end_date'].dt.tz_localize('US/Eastern')
# zacks_fc['filing_date'] = pd.to_datetime(zacks_fc['filing_date'])
# zacks_fc['filing_date'] = zacks_fc['filing_date'].dt.tz_localize('US/Eastern')
# zacks_fc = zacks_fc.sort_values('per_end_date').set_index('per_end_date')

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_60214/513894337.py:1: DtypeWarning: Columns (4,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  zacks_fc = pd.read_csv("../data/fundamental_data/ZACKS_FC.txt", delimiter=',')


,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,stock_based_compsn_qd,cash_flow_oper_activity_qd,net_change_prop_plant_equip_qd,comm_stock_div_paid_qd,pref_stock_div_paid_qd,tot_comm_pref_stock_div_qd,wavg_shares_out,wavg_shares_out_diluted,eps_basic_net,eps_diluted_net
0,YSTI,V.NTS,NaN,NANOTECH SECRTY,NaN,CND,2015-03-31,Q,NaN,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,48.751,48.751,NaN,NaN
1,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2006-12-31,A,NaN,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2007-12-31,A,NaN,2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2008-12-31,A,NaN,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CPG,3262Q,NaN,CMS ENERGY -G,NaN,USD,2009-12-31,A,NaN,2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
tickers_list = zacks_fc["ticker"].unique()

In [60]:

# Connect to WRDS
db = wrds.Connection()

# Define start and end dates
start_date = '1990-01-01'
end_date = '2025-01-01'



WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [103]:

query_crsp = f"""
    SELECT msf.*,
           msi.*,  -- Explicitly select columns from msi to avoid ambiguity
           msenames.ticker
    FROM crsp.msf AS msf
    LEFT JOIN crsp.msi AS msi ON msf.date = msi.date
    LEFT JOIN crsp.msenames AS msenames 
        ON msf.permno = msenames.permno 
        AND msenames.namedt <= msf.date 
        AND msf.date <= msenames.nameendt
    WHERE msf.date BETWEEN '{start_date}' AND '{end_date}'
"""
crsp_data = db.raw_sql(query_crsp, date_cols=['date'])

crsp_data.to_csv('../data/wrds_features_all.csv', index=False)
db.close()

In [4]:
crsp_data = pd.read_csv('../data/wrds_features_all.csv')

In [5]:
crsp_data.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,vwretx,ewretd,ewretx,sprtrn,spindx,totval,totcnt,usdval,usdcnt,ticker
0,00621210,10065,20023,0,1,6722.0,2009-02-27,6.7087,8.04,6.710,...,-0.103353,-0.107606,-0.109708,-0.109931,735.09,9.985858e+09,6710,1.116577e+10,6698,ADX
1,00621210,10065,20023,0,1,6722.0,2009-03-31,6.0620,7.78,7.522,...,0.083548,0.107474,0.104391,0.085405,797.87,1.073418e+10,6665,9.893487e+09,6658,ADX
2,00621210,10065,20023,0,1,6722.0,2009-04-30,7.4900,8.15,8.150,...,0.107632,0.192762,0.190929,0.093925,872.81,1.190017e+10,6635,1.071496e+10,6617,ADX
3,00621210,10065,20023,0,1,6722.0,2009-05-29,8.1000,8.57,8.440,...,0.065137,0.102329,0.100210,0.053081,919.14,1.274711e+10,6598,1.189937e+10,6585,ADX
4,00621210,10065,20023,0,1,6722.0,2009-06-30,8.2100,8.70,8.400,...,-0.004962,0.023921,0.021352,0.000196,919.32,1.274607e+10,6575,1.274197e+10,6545,ADX


In [6]:
crsp_data.dtypes

cusip        object
permno        int64
permco        int64
issuno        int64
hexcd         int64
hsiccd      float64
date         object
bidlo       float64
askhi       float64
prc         float64
vol         float64
ret         float64
bid         float64
ask         float64
shrout      float64
cfacpr      float64
cfacshr     float64
altprc      float64
spread      float64
altprcdt     object
retx        float64
date.1       object
vwretd      float64
vwretx      float64
ewretd      float64
ewretx      float64
sprtrn      float64
spindx      float64
totval      float64
totcnt        int64
usdval      float64
usdcnt        int64
ticker       object
dtype: object

In [7]:
crsp_data.dropna(subset=['ticker'], inplace=True)

In [8]:
crsp_data['mktcap'] = crsp_data['prc'].abs()*crsp_data['cfacpr'] * crsp_data['shrout'] * crsp_data['cfacshr']  # Market Cap
crsp_data['turnover'] = crsp_data['vol'] / crsp_data['shrout']  # Turnover
crsp_data['adjprc'] = crsp_data['prc']*crsp_data['cfacpr']


In [9]:
crsp_data['prc'].isna().sum()

np.int64(1816)

In [113]:
query = f"""
    SELECT date, rf 
    FROM ff.factors_monthly
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""

# Retrieve data
t_bill_data = db.raw_sql(query, date_cols=['date'])
t_bill_data.head()

,date,rf
0,1990-01-01,0.0057
1,1990-02-01,0.0057
2,1990-03-01,0.0064
3,1990-04-01,0.0069
4,1990-05-01,0.0068


In [127]:
len(crsp_data[crsp_data.ticker=='V.NTS'])

0

In [131]:
concat_list = []
for ticker in tqdm(tickers_list, desc="cleaning", unit="ticker"):
    temp = crsp_data[crsp_data.ticker==ticker]
    if len(temp)==0:
        continue
    temp = temp.drop_duplicates(subset=['date'],keep='last')
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.sort_values(by='date', ascending=True) 
    # print(temp.date)
    temp = pd.merge_asof(
        temp,
        t_bill_data,
        on='date',
        direction='backward'
    )
    concat_list.append(temp)
crsp_data = pd.concat(concat_list,ignore_index=True)   


cleaning: 100%|██████████| 16688/16688 [39:03<00:00,  7.12ticker/s]


In [149]:
# crsp_data.to_csv("../data/wrds_processed_features.csv")

In [14]:
crsp_data = pd.read_csv("../data/wrds_processed_features.csv")

In [ ]:
crsp_data

Sanity Check

In [140]:
unique_dates = sorted(crsp_data['date'].unique())


date_diffs = pd.Series([unique_dates[i+1] - unique_dates[i] for i in range(len(unique_dates) - 1)])

average_diff = date_diffs.mean()

print(f"Average difference between dates: {average_diff}")
pd.DataFrame(date_diffs).describe()

Average difference between dates: 30 days 10:28:55.560859188


,0
count,419
mean,30 days 10:28:55.560859188
std,1 days 11:56:03.733891209
min,28 days 00:00:00
25%,29 days 00:00:00
50%,31 days 00:00:00
75%,31 days 00:00:00
max,33 days 00:00:00


In [6]:
crsp_data.columns

Index(['Unnamed: 0', 'cusip', 'permno', 'permco', 'issuno', 'hexcd', 'hsiccd',
       'date', 'bidlo', 'askhi', 'prc', 'vol', 'ret', 'bid', 'ask', 'shrout',
       'cfacpr', 'cfacshr', 'altprc', 'spread', 'altprcdt', 'retx', 'date.1',
       'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn', 'spindx', 'totval',
       'totcnt', 'usdval', 'usdcnt', 'ticker', 'mktcap', 'turnover', 'adjprc',
       'momentum', 'short_term_reversal', 'long_term_reversal', '52wk_high',
       'price_rel_52wk_high', 'rf', 'beta', 'idvol', 'unexp_vol'],
      dtype='object')

In [148]:
crsp_data = crsp_data.drop(columns=['rf_x'])
crsp_data = crsp_data.rename(columns={"rf_y":"rf"})
crsp_data.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ticker,mktcap,turnover,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf
0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,0.4375,-0.31250,...,A,3980.31250,0.056685,-0.31250,NaN,0.625000,NaN,NaN,NaN,0.0057
1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,0.4375,0.37500,...,A,4776.37500,0.065243,0.37500,NaN,0.166667,NaN,NaN,NaN,0.0057
2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,-0.3750,-0.37500,...,A,4776.37500,0.049541,-0.37500,NaN,-0.200000,NaN,NaN,NaN,0.0064
3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,0.3750,-0.28125,...,A,3582.28125,0.055979,-0.28125,NaN,-0.000000,NaN,NaN,NaN,0.0069
4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,-0.3125,0.25000,...,A,3184.25000,0.085577,0.25000,NaN,0.250000,NaN,NaN,NaN,0.0068


In [150]:
del(concat_list)

In [151]:
def get_nan_count(return_series):
        return return_series.isna().sum()


In [152]:
temp = crsp_data.groupby('ticker')['ret'].apply(get_nan_count)
temp.sort_values(ascending=False)

ticker
RFIL    80
HIRE    77
SVA     72
STRP    48
GLMA    39
        ..
DDS      0
PHIO     0
PHLT     0
PHM      0
PTMN     0
Name: ret, Length: 10085, dtype: int64

In [15]:
def compute_momentum(ret):
    """Computes past 11-month cumulative return, skipping the most recent month."""
    
    return  np.exp(np.log(1+ret).rolling(11,min_periods=11).sum().shift(1))-1

crsp_data['momentum'] = crsp_data.groupby('permno')['ret'].transform(compute_momentum)


In [16]:
crsp_data['short_term_reversal'] = -crsp_data['ret'].shift(1)

def compute_long_term_reversal(ret):
    """Computes past 5-year cumulative return, skipping the most recent year."""
    
    return  np.exp(np.log(1+ret).rolling(60,min_periods=60).sum().shift(12))-1

# Compute Long-Term Reversal (-5-Year Return, lagged 1 year)
crsp_data['long_term_reversal'] = crsp_data.groupby('permno')['ret'].transform(compute_long_term_reversal)


def compute_52_week_high(adjprc):
    """Computes past 5-year cumulative return, skipping the most recent year."""
    return adjprc.rolling(12,min_periods=12).max().shift(12)

# Compute Price Relative to 52-Week High
crsp_data['52wk_high'] = crsp_data.groupby('permno')['adjprc'].transform(compute_52_week_high)
crsp_data['price_rel_52wk_high'] = crsp_data['adjprc'] / crsp_data['52wk_high']

In [17]:
crsp_data.isna().sum()

Unnamed: 0                   0
cusip                        0
permno                       0
permco                       0
issuno                       0
hexcd                        0
hsiccd                    1876
date                         0
bidlo                      519
askhi                      519
prc                        660
vol                        519
ret                       9343
bid                      65989
ask                      65990
shrout                       6
cfacpr                       6
cfacshr                      6
altprc                     513
spread                 1506096
altprcdt                     0
retx                      9343
date.1                       0
vwretd                       0
vwretx                       0
ewretd                       0
ewretx                       0
sprtrn                       0
spindx                       0
totval                       0
totcnt                       0
usdval                       0
usdcnt  

In [10]:
crsp_data.head()

,Unnamed: 0,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,...,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf,beta,idvol,unexp_vol
0,0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,0.4375,...,-0.31250,NaN,NaN,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
1,1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,0.4375,...,0.37500,NaN,0.166667,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
2,2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,-0.3750,...,-0.37500,NaN,-0.200000,NaN,NaN,NaN,0.0064,NaN,NaN,NaN
3,3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,0.3750,...,-0.28125,NaN,-0.000000,NaN,NaN,NaN,0.0069,NaN,NaN,NaN
4,4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,-0.3125,...,0.25000,NaN,0.250000,NaN,NaN,NaN,0.0068,NaN,NaN,NaN


In [18]:
crsp_data.to_csv("../data/crsp_ipsa_features.csv")

array([60409, 11334, 83438, ..., 15646, 18418, 91205], shape=(13092,))

In [11]:
crsp_data.ticker.nunique()

10085

In [12]:
crsp_data['prc'] = abs(crsp_data['prc'] )

In [159]:
# def compute_rf(group):
#     # Merge with t_bill_data
#     group['date'] = pd.to_datetime(group['date'])
#     print(group.ticker.iloc[0])
#     if group.ticker.iloc[0] == 'SCTTA':
#         print(group.date)
#     # print(group.date)
#     group = group[['date']]
#     merged = pd.merge_asof(
#         group,
#         t_bill_data,
#         on='date',
#         direction='backward'
#     )
#     return merged['rf']
# rf_rates_1m  = crsp_data.groupby('permno').apply(compute_rf).reset_index(level=0, drop=True)
# crsp_data = crsp_data.merge(rf_rates_1m, left_index=True, right_index=True, how='left')

In [13]:
def compute_rolling_metrics(group, window=12):
    """
    Computes rolling beta, idiosyncratic volatility, and unexplained volume for a stock.
    
    Parameters:
        group (DataFrame): Data with 'ret', 'vwretd', 'rf', and 'vol' columns
        window (int): Rolling window size in months
    
    Returns:
        DataFrame: Rolling metrics with NaN for initial window-1 periods
    """
    # Convert to numeric if not already
    ret = pd.to_numeric(group['ret'], errors='coerce')
    rf = pd.to_numeric(group['rf'], errors='coerce')
    vwretd = pd.to_numeric(group['vwretd'], errors='coerce')
    volume = pd.to_numeric(group['vol'], errors='coerce')
    
    # If insufficient data, return DataFrame of NaNs
    if len(group) < window:
        return pd.DataFrame({
            'beta': np.nan,
            'idvol': np.nan,
            'unexp_vol': np.nan
        }, index=group.index)
    
    metrics_list = []
    
    # Calculate rolling metrics
    for i in range(window, len(group) + 1):
        try:
            # Prepare return data
            temp_ret = np.asarray(ret.iloc[i - window:i] - rf.iloc[i-window:i])
            temp_vwretd = np.asarray(vwretd.iloc[i - window:i] - rf.iloc[i-window:i])
            
            # Create design matrix and handle missing values
            X = sm.add_constant(temp_vwretd)
            y = temp_ret
            
            metrics = {}
            
            # Only compute if we have sufficient non-NaN values
            if len(X) >= window-2 and len(y) >= window-2:  # Allow for some missing data
                # Fit market model
                model = sm.OLS(y, X, missing='drop').fit()
                
                # Calculate beta
                metrics['beta'] = model.params[1]
                
                # Calculate idiosyncratic volatility (standard deviation of residuals)
                metrics['idvol'] = np.std(model.resid)
                
                # Calculate unexplained volume if volume data is available
                if volume is not None:
                    temp_vol = volume.iloc[i - window:i]
                    if len(temp_vol) >= window-2:
                        # Detrend volume using log and first difference
                        log_vol = np.log(temp_vol)
                        vol_change = log_vol.diff()
                        
                        # Regress volume changes on absolute market returns
                        X_vol = sm.add_constant(np.abs(temp_vwretd))
                        # print(X_vol.dtype,type(vol_change))
                        vol_model = sm.OLS(np.asarray(vol_change[1:]), X_vol[1:], missing='drop').fit()
                        
                        # Standard deviation of volume regression residuals
                        metrics['unexp_vol'] = np.std(vol_model.resid)
                    else:
                        metrics['unexp_vol'] = np.nan
                else:
                    metrics['unexp_vol'] = np.nan
            else:
                metrics = {
                    'beta': np.nan,
                    'idvol': np.nan,
                    'unexp_vol': np.nan
                }
                
        except (ValueError, np.linalg.LinAlgError):
            metrics = {
                'beta': np.nan,
                'idvol': np.nan,
                'unexp_vol': np.nan
            }
            
        metrics_list.append(metrics)
    
    # Create full DataFrame with NaN padding at start
    nan_rows = [{
        'beta': np.nan,
        'idvol': np.nan,
        'unexp_vol': np.nan
    } for _ in range(window - 1)]
    
    full_df = pd.DataFrame(
        nan_rows + metrics_list,
        index=group.index
    )
    
    return full_df


result = crsp_data.groupby('permno').apply(compute_rolling_metrics).reset_index(level=0, drop=True)

crsp_data = crsp_data.merge(result, left_index=True, right_index=True, how='left')

/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1704: RuntimeWarning: invalid value encountered in subtract
  return self.model.endog - self.model.predict(


KeyboardInterrupt: 

In [256]:
crsp_data[crsp_data.ticker=="AAPL"]

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,long_term_reversal,52wk_high,price_rel_52wk_high,rf,beta_x,idvol_x,unexp_vol_x,beta_y,idvol_y,unexp_vol_y
3787,03783310,14593,7,8,3,3571.0,1990-01-31,32.37500,38.00000,34.000,...,NaN,NaN,NaN,0.0057,NaN,NaN,NaN,NaN,NaN,NaN
3788,03783310,14593,7,8,3,3571.0,1990-02-28,33.00000,35.00000,34.000,...,NaN,NaN,NaN,0.0057,NaN,NaN,NaN,NaN,NaN,NaN
3789,03783310,14593,7,8,3,3571.0,1990-03-30,33.75000,42.37500,40.250,...,NaN,NaN,NaN,0.0064,NaN,NaN,NaN,NaN,NaN,NaN
3790,03783310,14593,7,8,3,3571.0,1990-04-30,38.75000,43.75000,39.375,...,NaN,NaN,NaN,0.0069,NaN,NaN,NaN,NaN,NaN,NaN
3791,03783310,14593,7,8,3,3571.0,1990-05-31,39.50000,42.62500,41.250,...,NaN,NaN,NaN,0.0068,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,03783310,14593,7,8,3,3571.0,2024-08-30,207.23000,229.78999,229.000,...,2.450649,1700.16004,0.134693,0.0048,NaN,NaN,NaN,0.976685,0.052552,0.170252
4203,03783310,14593,7,8,3,3571.0,2024-09-30,216.32001,233.00000,233.000,...,2.170981,1700.16004,0.137046,0.004,NaN,NaN,NaN,0.763847,0.050452,0.167586
4204,03783310,14593,7,8,3,3571.0,2024-10-31,221.69000,236.48000,225.910,...,2.262259,1700.16004,0.132876,0.0039,NaN,NaN,NaN,0.901130,0.050927,0.188185
4205,03783310,14593,7,8,3,3571.0,2024-11-29,222.00999,237.33000,237.330,...,3.434618,1700.16004,0.139593,0.004,NaN,NaN,NaN,0.663048,0.049729,0.188319


In [257]:
crsp_data = crsp_data.drop(columns=["beta_x",	"idvol_x",	"unexp_vol_x"])
crsp_data = crsp_data.rename(columns={"beta_y":	"beta","idvol_y": "idvol",	"unexp_vol_y":"unexp_vol"})
crsp_data[crsp_data.ticker=="AAPL"]

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,adjprc,momentum,short_term_reversal,long_term_reversal,52wk_high,price_rel_52wk_high,rf,beta,idvol,unexp_vol
3787,03783310,14593,7,8,3,3571.0,1990-01-31,32.37500,38.00000,34.000,...,3808.00,NaN,-0.143652,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
3788,03783310,14593,7,8,3,3571.0,1990-02-28,33.00000,35.00000,34.000,...,3808.00,NaN,0.035461,NaN,NaN,NaN,0.0057,NaN,NaN,NaN
3789,03783310,14593,7,8,3,3571.0,1990-03-30,33.75000,42.37500,40.250,...,4508.00,NaN,-0.003235,NaN,NaN,NaN,0.0064,NaN,NaN,NaN
3790,03783310,14593,7,8,3,3571.0,1990-04-30,38.75000,43.75000,39.375,...,4410.00,NaN,-0.183824,NaN,NaN,NaN,0.0069,NaN,NaN,NaN
3791,03783310,14593,7,8,3,3571.0,1990-05-31,39.50000,42.62500,41.250,...,4620.00,NaN,0.021739,NaN,NaN,NaN,0.0068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,03783310,14593,7,8,3,3571.0,2024-08-30,207.23000,229.78999,229.000,...,229.00,0.186703,-0.054411,2.450649,1700.16004,0.134693,0.0048,0.976685,0.052552,0.170252
4203,03783310,14593,7,8,3,3571.0,2024-09-30,216.32001,233.00000,233.000,...,233.00,0.344220,-0.032286,2.170981,1700.16004,0.137046,0.004,0.763847,0.050452,0.167586
4204,03783310,14593,7,8,3,3571.0,2024-10-31,221.69000,236.48000,225.910,...,225.91,0.371223,-0.017467,2.262259,1700.16004,0.132876,0.0039,0.901130,0.050927,0.188185
4205,03783310,14593,7,8,3,3571.0,2024-11-29,222.00999,237.33000,237.330,...,237.33,0.193745,0.030429,3.434618,1700.16004,0.139593,0.004,0.663048,0.049729,0.188319


In [278]:
crsp_data.to_csv("../data/wrds_processed_features.csv")

In [19]:
# Fundamental data

In [26]:
import pandas as pd
import numpy as np
import wrds
import datetime
import sys
import argparse

def get_compustat_quarterly_for_permnos(permnos, start_date, end_date, variables=None):
    """
    Pull quarterly Compustat data from WRDS for a specific list of PERMNOs
    
    Parameters:
    -----------
    permnos : list
        List of PERMNOs to retrieve data for
    start_date : str
        Start date in format 'YYYY-MM-DD'
    end_date : str
        End date in format 'YYYY-MM-DD'
    variables : list, optional
        List of Compustat variables to retrieve. If None, default set of variables will be used.
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the requested Compustat quarterly data
    """
    
    # Connect to WRDS
    conn = wrds.Connection()
    
    # Check if the linking table exists
    link_table = None
    possible_tables = ['crsp.ccmxpf_linktable', 'comp.ccmxpf_linktable', 'crsp.ccmxpf_lnkhist', 'comp.ccmxpf_lnkhist']
    
    for table in possible_tables:
        try:
            # Try a simple query to check if the table exists
            test_query = f"SELECT * FROM {table} LIMIT 1"
            conn.raw_sql(test_query)
            link_table = table
            print(f"Found linking table: {link_table}")
            break
        except:
            continue
    
    if link_table is None:
        # If we can't find the linking table, try to directly query information schema
        try:
            schema_query = """
            SELECT table_schema, table_name 
            FROM information_schema.tables 
            WHERE table_name LIKE '%ccm%' AND table_name LIKE '%link%'
            """
            schema_results = conn.raw_sql(schema_query)
            if not schema_results.empty:
                link_table = f"{schema_results.iloc[0]['table_schema']}.{schema_results.iloc[0]['table_name']}"
                print(f"Found linking table: {link_table}")
        except:
            pass
    
    if link_table is None:
        raise ValueError("Could not find CRSP-Compustat linking table. Please check your WRDS subscription.")
    
    # Default set of variables if none provided
    if variables is None:
        variables = [
            'fundq.gvkey',           # Global company key
            'fundq.datadate',        # Data date
            'fundq.fyearq',          # Fiscal year for quarter
            'fundq.fqtr',            # Fiscal quarter
            'fundq.tic',             # Ticker symbol
            'fundq.cusip',           # CUSIP
            'fundq.conm',            # Company name
            'fundq.revtq',           # Revenue - Total
            'fundq.niq',             # Net Income (Loss)
            'fundq.atq',             # Assets - Total
            'fundq.epsfxq',          # Earnings Per Share (Diluted) - Excluding Extraordinary Items
            'fundq.cheq',            # Cash and Short-Term Investments
            'fundq.dlttq',           # Long-Term Debt - Total
            'fundq.cogsq',           # Cost of Goods Sold
            'fundq.oibdpq',          # Operating Income Before Depreciation
            'fundq.cshoq',           # Common Shares Outstanding
            'fundq.xrdq'             # Research and Development Expense
        ]
    
    # Get the column names for PERMNO in the linking table
    # Different tables might use different column names (lpermno, permno, etc.)
    permno_col = None
    try:
        test_query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{link_table.split('.')[-1]}' AND column_name LIKE '%permno%'"
        col_results = conn.raw_sql(test_query)
        if not col_results.empty:
            permno_col = col_results.iloc[0]['column_name']
            print(f"Found PERMNO column: {permno_col}")
    except:
        # Try common permno column names
        for col in ['lpermno', 'permno']:
            try:
                test_query = f"SELECT {col} FROM {link_table} LIMIT 1"
                conn.raw_sql(test_query)
                permno_col = col
                print(f"Found PERMNO column: {permno_col}")
                break
            except:
                continue
    
    if permno_col is None:
        permno_col = 'lpermno'  # Default to lpermno if we can't determine it
    
    # Add permno to variables
    variables.append(f"{link_table}.{permno_col} as permno")
    
    # Create chunks of PERMNOs to avoid query size limitations
    chunk_size = 1000
    permno_chunks = [permnos[i:i + chunk_size] for i in range(0, len(permnos), chunk_size)]
    
    # Initialize empty DataFrame for results
    all_data = pd.DataFrame()
    
    # Process each chunk
    for i, chunk in enumerate(permno_chunks):
        print(f"Processing chunk {i+1} of {len(permno_chunks)} ({len(chunk)} PERMNOs)...")
        
        permno_list = ', '.join(map(str, chunk))
        
        # Create SQL query with CRSP link and PERMNO filter
        query = f"""
        SELECT {', '.join(variables)}
        FROM comp.fundq 
        LEFT JOIN {link_table}
        ON fundq.gvkey = {link_table}.gvkey
        WHERE fundq.datadate BETWEEN '{start_date}' AND '{end_date}'
        AND {link_table}.linktype IN ('LU', 'LC')
        AND {link_table}.linkprim IN ('P', 'C')
        AND (fundq.datadate >= {link_table}.linkdt)
        AND ({link_table}.linkenddt IS NULL OR fundq.datadate <= {link_table}.linkenddt)
        AND {link_table}.{permno_col} IN ({permno_list})
        """
        
        try:
            # Execute query
            chunk_data = conn.raw_sql(query)
            
            # Append to results
            all_data = pd.concat([all_data, chunk_data])
        except Exception as e:
            print(f"Error executing query: {e}")
            
            # Try a simpler approach as fallback
            print("Trying alternative approach...")
            try:
                # First get gvkeys for the PERMNOs
                gvkey_query = f"""
                SELECT DISTINCT gvkey, {permno_col}
                FROM {link_table}
                WHERE {permno_col} IN ({permno_list})
                """
                gvkeys_df = conn.raw_sql(gvkey_query)
                
                if not gvkeys_df.empty:
                    gvkey_list = ', '.join([f"'{gvkey}'" for gvkey in gvkeys_df['gvkey'].unique()])
                    
                    # Then get the fundamentals data for those gvkeys
                    fund_query = f"""
                    SELECT {', '.join([v for v in variables if 'fundq.' in v])}
                    FROM comp.fundq
                    WHERE gvkey IN ({gvkey_list})
                    AND datadate BETWEEN '{start_date}' AND '{end_date}'
                    """
                    fund_data = conn.raw_sql(fund_query)
                    
                    # Merge with the permno information
                    merged_data = pd.merge(
                        fund_data, 
                        gvkeys_df, 
                        on='gvkey', 
                        how='left'
                    )
                    
                    # Rename permno column
                    merged_data = merged_data.rename(columns={permno_col: 'permno'})
                    
                    # Append to results
                    all_data = pd.concat([all_data, merged_data])
            except Exception as e2:
                print(f"Alternative approach also failed: {e2}")
                print("Please check your WRDS subscription and table names.")
    
    # Close connection
    conn.close()
    
    # Clean up column names
    all_data.columns = [col.split('.')[-1] for col in all_data.columns]
    
    return all_data

def clean_compustat_data(data):
    """
    Clean and prepare Compustat data for analysis
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Raw Compustat data
        
    Returns:
    --------
    pandas.DataFrame
        Cleaned Compustat data
    """
    
    # Check if we have data
    if data.empty:
        print("Warning: No data retrieved")
        return data
    
    # Convert date columns
    if 'datadate' in data.columns:
        data['datadate'] = pd.to_datetime(data['datadate'])
    
    # Create year-quarter column
    if all(col in data.columns for col in ['fyearq', 'fqtr']):
        data['year_quarter'] = data['fyearq'].astype(str) + 'Q' + data['fqtr'].astype(str)
    
    # Replace missing values with NaN
    numeric_cols = ['revtq', 'niq', 'atq', 'epsfxq', 'cheq', 'dlttq', 'cogsq', 'oibdpq', 'cshoq', 'xrdq', 'permno']
    for col in numeric_cols:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Calculate additional metrics only if required columns exist
    if all(col in data.columns for col in ['revtq', 'cogsq']):
        data['gross_profit'] = data['revtq'] - data['cogsq']
        data['gross_margin'] = data['gross_profit'] / data['revtq']
    
    if all(col in data.columns for col in ['niq', 'revtq']):
        data['profit_margin'] = data['niq'] / data['revtq']
    
    if all(col in data.columns for col in ['dlttq', 'atq']):
        data['debt_to_assets'] = data['dlttq'] / data['atq']
    
    return data

def load_permnos_from_file(filename):
    """
    Load PERMNOs from a file
    
    Parameters:
    -----------
    filename : str
        Path to file containing PERMNOs
        
    Returns:
    --------
    list
        List of PERMNOs
    """
    try:
        # Try to read as CSV
        df = pd.read_csv(filename)
        
        # Check if 'permno' column exists
        if 'permno' in df.columns:
            permnos = df['permno'].unique().tolist()
        # If not, try to use the first column
        else:
            permnos = df.iloc[:, 0].unique().tolist()
            
        # Convert to integers
        permnos = [int(p) for p in permnos if not pd.isna(p)]
        
        return permnos
    except:
        # Try to read as text file with one PERMNO per line
        try:
            with open(filename, 'r') as f:
                permnos = [int(line.strip()) for line in f if line.strip().isdigit()]
            return permnos
        except:
            raise ValueError(f"Could not read PERMNOs from {filename}")

def get_compustat_for_permno_list(permno_list, start_date='2020-01-01', end_date='2023-12-31', output_file='compustat_data.csv'):
    """
    Get Compustat data for a list of PERMNOs
    
    Parameters:
    -----------
    permno_list : list
        List of PERMNOs to retrieve data for
    start_date : str, optional
        Start date in format 'YYYY-MM-DD'
    end_date : str, optional
        End date in format 'YYYY-MM-DD'
    output_file : str, optional
        Name of output file
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the Compustat quarterly data
    """
    # Convert PERMNOs to integers if needed
    permnos = [int(p) for p in permno_list if str(p).strip().isdigit()]
    
    print(f"Retrieving Compustat data for {len(permnos)} PERMNOs...")
    raw_data = get_compustat_quarterly_for_permnos(permnos, start_date, end_date)
    
    print("Cleaning and processing data...")
    clean_data = clean_compustat_data(raw_data)
    
    if output_file and not clean_data.empty:
        print(f"Saving data to {output_file}...")
        clean_data.to_csv(output_file, index=False)
    
    # Display summary statistics
    if not clean_data.empty:
        print("\nData Summary:")
        print(f"Number of companies (GVKEYs): {clean_data['gvkey'].nunique()}")
        print(f"Number of unique PERMNOs: {clean_data['permno'].nunique()}")
        print(f"Date range: {clean_data['datadate'].min()} to {clean_data['datadate'].max()}")
        print(f"Total observations: {len(clean_data)}")
    else:
        print("No data was retrieved. Please check your WRDS subscription and table access.")
    
    return clean_data

# Example usage
# if __name__ == "__main__":
#     # Example input directly in code
#     # Replace with your actual list of PERMNOs
#     sample_permnos = [10107, 10145, 11308, 11850, 12060, 12490, 13901, 14322, 14542]
    
#     # Get data for these PERMNOs
#     data = get_compustat_for_permno_list(sample_permnos)

In [27]:
permnos = list(crsp_data.permno.unique())

In [28]:
start_date = '1990-01-01'
end_date = '2025-01-01'
raw_data = get_compustat_quarterly_for_permnos(permnos,start_date, end_date)

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Found linking table: crsp.ccmxpf_linktable
Found PERMNO column: lpermno
Processing chunk 1 of 14 (1000 PERMNOs)...
Processing chunk 2 of 14 (1000 PERMNOs)...
Processing chunk 3 of 14 (1000 PERMNOs)...
Processing chunk 4 of 14 (1000 PERMNOs)...
Processing chunk 5 of 14 (1000 PERMNOs)...
Processing chunk 6 of 14 (1000 PERMNOs)...
Processing chunk 7 of 14 (1000 PERMNOs)...
Processing chunk 8 of 14 (1000 PERMNOs)...
Processing chunk 9 of 14 (1000 PERMNOs)...
Processing chunk 10 of 14 (1000 PERMNOs)...
Processing chunk 11 of 14 (1000 PERMNOs)...
Processing chunk 12 of 14 (1000 PERMNOs)...
Processing chunk 13 of 14 (1000 PERMNOs)...
Processing chunk 14 of 14 (92 PERMNOs)...


In [29]:
raw_data.head()

,gvkey,datadate,fyearq,fqtr,tic,cusip,conm,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,xrdq,permno
0,001004,1990-02-28,1989,3,AIR,000361105,AAR CORP,112.278,6.109,383.79,0.38,6.048,72.606,84.405,14.251,16.07,<NA>,54594.0
1,001004,1990-05-31,1989,4,AIR,000361105,AAR CORP,119.396,6.224,388.521,0.39,4.044,72.329,92.269,13.137,16.082,<NA>,54594.0
2,001004,1990-08-31,1990,1,AIR,000361105,AAR CORP,116.092,6.697,385.752,0.42,2.814,71.806,86.875,15.4,16.086,<NA>,54594.0
3,001004,1990-11-30,1990,2,AIR,000361105,AAR CORP,115.808,0.126,382.978,0.01,4.331,71.769,93.257,8.871,15.879,<NA>,54594.0
4,001004,1991-02-28,1990,3,AIR,000361105,AAR CORP,117.82,3.977,383.354,0.25,5.019,69.02,91.708,11.344,15.891,<NA>,54594.0


In [157]:
# raw_data.to_csv("../data/computstat_features.csv")
raw_data = pd.read_csv("../data/computstat_features.csv")

In [158]:
# for i in raw_data.permno.unique():
#     if int(i) not in permnos:
#         print(i)


In [159]:
raw_data = raw_data.rename(columns = {"datadate": "date"})
raw_data['date'] = pd.to_datetime(raw_data['date'])
raw_data["permno"] = raw_data.permno.astype(int)

In [160]:
from datetime import timedelta

In [161]:
dummy_merge = crsp_data[['date','permno']]
dummy_merge.head()

,date,permno
0,1990-01-31,60409
1,1990-02-28,60409
2,1990-03-30,60409
3,1990-04-30,60409
4,1990-05-31,60409


In [162]:
temp = raw_data[raw_data.permno==54594]
for col in temp.columns:
    if col not in ['date','permno']:
        temp[col] = temp[col].shift(1)
temp.head()

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_2659/2456250993.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[col] = temp[col].shift(1)


,Unnamed: 0,gvkey,date,fyearq,fqtr,tic,cusip,conm,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,xrdq,permno
0,NaN,NaN,1990-02-28,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54594
1,0.0,1004.0,1990-05-31,1989.0,3.0,AIR,000361105,AAR CORP,112.278,6.109,383.790,0.38,6.048,72.606,84.405,14.251,16.070,NaN,54594
2,1.0,1004.0,1990-08-31,1989.0,4.0,AIR,000361105,AAR CORP,119.396,6.224,388.521,0.39,4.044,72.329,92.269,13.137,16.082,NaN,54594
3,2.0,1004.0,1990-11-30,1990.0,1.0,AIR,000361105,AAR CORP,116.092,6.697,385.752,0.42,2.814,71.806,86.875,15.400,16.086,NaN,54594
4,3.0,1004.0,1991-02-28,1990.0,2.0,AIR,000361105,AAR CORP,115.808,0.126,382.978,0.01,4.331,71.769,93.257,8.871,15.879,NaN,54594


In [163]:
concat_list = []
for permno in tqdm(raw_data.permno.unique(), desc="processing", unit="ticker"):
    t1 = dummy_merge[dummy_merge.permno==permno]
    t1 = t1.sort_values(by='date', ascending=True) 
    temp = raw_data[raw_data.permno==permno]
    # temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.sort_values(by='date', ascending=True) 
    for col in temp.columns:
        if col not in ['date','permno']:
            temp[col] = temp[col].shift(1)
    
 
    # print(temp.date)
    x = pd.merge_asof(
        t1,
        temp,
        on='date',
        direction='nearest'
    )
    x = x.ffill()
    concat_list.append(x)
final_df = pd.concat(concat_list,ignore_index=True)   

processing: 100%|██████████| 12197/12197 [00:29<00:00, 415.34ticker/s]


In [165]:
len(final_df)

1534439

In [166]:
final_df = final_df.rename(columns={'permno_x':'permno'})

In [167]:
final_df = final_df.drop(['Unnamed: 0','fyearq', 'fqtr', 'tic', 'cusip', 'conm','permno_y'], axis=1)

In [168]:
final_df

,date,permno,gvkey,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,xrdq
0,1990-01-31,54594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-02-28,54594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-03-30,54594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-04-30,54594,1004.0,112.278,6.109,383.790,0.38,6.048,72.606,84.405,14.251,16.070,NaN
4,1990-05-31,54594,1004.0,112.278,6.109,383.790,0.38,6.048,72.606,84.405,14.251,16.070,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534434,1999-10-29,93316,11768.0,6.709,-0.023,39.890,-0.02,22.811,3.231,4.251,0.362,2.408,1.853
1534435,1999-11-30,93316,11768.0,6.002,0.703,40.819,0.29,23.767,3.074,3.754,0.480,2.407,1.853
1534436,1999-12-31,93316,11768.0,6.002,0.703,40.819,0.29,23.767,3.074,3.754,0.480,2.407,1.853
1534437,2000-01-31,93316,11768.0,6.002,0.703,40.819,0.29,23.767,3.074,3.754,0.480,2.407,1.853


In [169]:
final_list = crsp_data.merge(final_df,how='left',left_on=['date','permno'],right_on=['date','permno'])

In [170]:
len(crsp_data)

1562322

In [171]:
final_list.isna().sum()[-10:]

revtq     229136
niq        82658
atq        83191
epsfxq     84891
cheq       83940
dlttq      84172
cogsq      83074
oibdpq    161613
cshoq      66306
xrdq      749945
dtype: int64

In [172]:
final_list = final_list.drop(columns=['xrdq'])
final_list.head()

,Unnamed: 0,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,...,gvkey,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq
0,0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,0.4375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,0.4375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,-0.3750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,0.3750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,-0.3125,...,1506.0,5.126,0.017,4.172,0.0,0.0,0.0,3.133,0.084,12.737


In [173]:
final_list[final_list["ticker"]=="AAPL"]

,Unnamed: 0,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,...,gvkey,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq
3787,3787,03783310,14593,7,8,3,3571.0,1990-01-31,32.37500,38.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3788,3788,03783310,14593,7,8,3,3571.0,1990-02-28,33.00000,35.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3789,3789,03783310,14593,7,8,3,3571.0,1990-03-30,33.75000,42.37500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3790,3790,03783310,14593,7,8,3,3571.0,1990-04-30,38.75000,43.75000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3791,3791,03783310,14593,7,8,3,3571.0,1990-05-31,39.50000,42.62500,...,1690.0,1346.202,131.815,2775.67,1.04,995.839,0.0,565.299,243.463,123.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,4202,03783310,14593,7,8,3,3571.0,2024-08-30,207.23000,229.78999,...,1690.0,85777.000,21448.000,331612.00,1.40,61801.000,86196.0,43249.000,28202.000,15222.259
4203,4203,03783310,14593,7,8,3,3571.0,2024-09-30,216.32001,233.00000,...,1690.0,85777.000,21448.000,331612.00,1.40,61801.000,86196.0,43249.000,28202.000,15222.259
4204,4204,03783310,14593,7,8,3,3571.0,2024-10-31,221.69000,236.48000,...,1690.0,85777.000,21448.000,331612.00,1.40,61801.000,86196.0,43249.000,28202.000,15222.259
4205,4205,03783310,14593,7,8,3,3571.0,2024-11-29,222.00999,237.33000,...,1690.0,94930.000,14736.000,364980.00,0.97,65171.000,96548.0,48140.000,32502.000,15116.786


In [174]:
final_list.to_csv("../data/ipca_features_final.csv")